In [ ]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import openai
import os
import docx
import json
from pprint import pprint
from tqdm import tqdm
import torch, gc

from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from extraction.docx_module import DocxHandler
from retriever.milvus_hybrid import MilvusHybridSearchHandler
from rag.llm_module_radmir import LLMManager
from main import QASystem

In [3]:
# очистка кэша
gc.collect()
torch.cuda.empty_cache()

docx_path = 'data/train/documentation.docx'       # путь к .docx файлу для обработки
csv_path = 'data/train/rag_df_processed.csv'      # путь к предварительно обработанному .csv файлу
collection_name = 'sila_collection_test3'         # название коллекции Milvus для dense search
embedding_model = "deepvk/USER-bge-m3"            # модель векторизации
connection_uri = 'milvus.db'                      # URI для подключения к базе данных Milvus
num_docs = 7                                      # количество документов для извлечения по запросу
device = 'cuda'                                   # устройство для создания векторных представлений (например, 'cuda' или 'cpu')

# определение системного класса
qa_system = QASystem(
    docx_path=docx_path,
    csv_path=csv_path,
    collection_name=collection_name,
    embedding_model=embedding_model,
    connection_uri=connection_uri,
    num_docs=num_docs,
    device=device
)

/workspace/october/main.py:46: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.hf_embeddings = HuggingFaceEmbeddings(


Collection 'sila_collection_test3' loaded.
INFO 10-27 05:46:41 gptq_marlin.py:107] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 10-27 05:46:41 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='qilowoq/Vikhr-Nemo-12B-Instruct-R-21-09-24-4Bit-GPTQ', speculative_config=None, tokenizer='qilowoq/Vikhr-Nemo-12B-Instruct-R-21-09-24-4Bit-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_e

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-27 05:46:45 model_runner.py:1067] Loading model weights took 8.0405 GB
INFO 10-27 05:46:47 gpu_executor.py:122] # GPU blocks: 12193, # CPU blocks: 1638
INFO 10-27 05:46:47 gpu_executor.py:126] Maximum concurrency for 8000 tokens per request: 24.39x
INFO 10-27 05:46:50 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-27 05:46:50 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-27 05:47:09 model_runner.py:1523] Graph capturing finished in 19 secs.


In [4]:
question = "Как редактировать данные в УЗ?"
answer = qa_system.answer_question(question, use_hyde=True)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.63s/it, est. speed input: 368.69 toks/s, output: 60.56 toks/s]


Query: Как редактировать данные в УЗ? | **Управление учетными записями в Системе управления безопасностью конфигураций ПО (СУБКК)**

Для редактирования данных в учетной записи в Системе управления безопасностью конфигураций ПО (СУБКК), необходимо войти в систему с правами администратора. После авторизации, в панели управления доступна функция редактирования учетных записей. Каждая учетная запись имеет уникальный идентификатор и статус активности, который можно изменить в соответствии с требованиями безопасности и политиками организации. Редактирование данных включает изменение личной информации пользователя, его ролей и привилегий в системе.


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, est. speed input: 2969.49 toks/s, output: 5.79 toks/s]


relevant indexes: [0]
[{'id': 0, 'main_text': 'Название секции: Работа в системе / Управление учетными записями |  Текст секции: Управление учетными записями (далее – УЗ) позволяет использовать их для доступа к ресурсам. Система содержит следующую информацию об УЗ: наименование; логин; пароль; отметка об использовании. Отметка об использовании показывает, используется ли УЗ в каком-либо ресурсе. С УЗ можно осуществлять следующие действия: создавать; редактировать; удалять. Для управления УЗ требуется выбрать пункт «Администрирование» – «Учетные записи ПО» в меню на странице системы. / Управление учетными записями - Создание УЗ |   Текст секции: Для создания УЗ: Нажмите кнопку «+ Создать» и дождитесь появления окна создания УЗ. Введите наименование УЗ. Введите логин УЗ. Введите пароль УЗ. Нажмите кнопку «Сохранить». Убедитесь, что созданная УЗ появилась в списке УЗ. / Управление учетными записями - Редактирование УЗ |   Текст секции: Для редактирования данных в УЗ: Нажмите на УЗ, котору

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.84s/it, est. speed input: 239.50 toks/s, output: 61.73 toks/s]


generated_answer Для редактирования данных в учетной записи (УЗ) вам необходимо выполнить следующие шаги:

1. Нажмите на учетную запись, которую вы хотите отредактировать, в списке учетных записей.
2. Нажмите на кнопку редактирования, которая обозначена как "Рисунок 4" в карточке выбранной учетной записи.
3. Внесите необходимые изменения в данные учетной записи.
4. Нажмите кнопку "Сохранить", чтобы сохранить внесенные изменения.

Обратите внимание, что перед удалением или редактированием учетной записи, которая используется в каких-либо ресурсах, убедитесь, что это не приведет к потере доступа к этим ресурсам.
paths_with_titles [{'path': 'data/extracted_images\\image_4.png', 'description': 'Рисунок 4. Редактирование УЗ'}, {'path': 'data/extracted_images\\image_5.png', 'description': 'Рисунок 5. Удаление УЗ'}]


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.18it/s, est. speed input: 1292.93 toks/s, output: 41.97 toks/s]

llm_photos_ids {"relevant_doc_ids": [0]}


In [5]:
print(answer[0])

Для редактирования данных в учетной записи (УЗ) вам необходимо выполнить следующие шаги:

1. Нажмите на учетную запись, которую вы хотите отредактировать, в списке учетных записей.
2. Нажмите на кнопку редактирования, которая обозначена как "Рисунок 4" в карточке выбранной учетной записи.
3. Внесите необходимые изменения в данные учетной записи.
4. Нажмите кнопку "Сохранить", чтобы сохранить внесенные изменения.

Обратите внимание, что перед удалением или редактированием учетной записи, которая используется в каких-либо ресурсах, убедитесь, что это не приведет к потере доступа к этим ресурсам.


#### Создание файла со всеми ответам (QA_all.docx)

In [ ]:
all_questions = [
    "С чего начать работу?",
    "Какой первый шаг?",
    "Как провести аудит?",
    "Что такое «Профиль»?",
    "Как создать профиль?",
    "Как создать требование?",
    "Что такое применимость профиля? Применимость требования? Могут ли применимости не совпадать?",
    "Как создать ресурс? Где взять IP-адрес ресурса?",
    "Зачем нужен ресурс оффлайн?",
    "Не создается ресурс, что делать?",
    "Как включить шлюз?",
    "В области аудита не виден профиль, почему?",
    "Почему не отображается ресурс при создании области аудита?",
    "Почему в требовании во вкладке «Сбор конфигурации» не отображается созданный ресурс?",
    "Что такое «Шаблон»?",
    "Выполнен аудит, почему статус протокола остался «В работе»?",
    "Модели ПО для чего нужны? Кто их создает?",
    "Как работать с Моделью ПО?",
    "Модели ПО и их связь с программной топологией ресурса",
    "Учетные записи ПО и их связь с программной топологией ресурса",
    "Закончилась лицензия КК – как продлить?",
    "Можно ли редактировать профиль «Активный»?",
    "Можно ли редактировать профиль «Архивный»?",
    "Можно ли создать копию профиля?",
    "Можно ли выгрузить скрипты выполненного протокола?",
    "Сколько максимально можно создать ресурсов?",
    "Сколько максимально можно создать профилей?",
    "Сколько максимально можно создать требований в профиле?",
    "Возможные ошибки при аудите?",
    "Статусы проверок в протоколе: «соответствует», «не соответствует», «в работе», «не применимо»",
    "Создано 2 блока применимости требования, выбираю требование, в нем не отображаются заполненные поля скрипта, почему?",
    "Не могу удалить профиль, почему?",
    "Можно ли создать свой шаблон и выгрузить его?",
    "Где ознакомиться с документацией на КК?",
    "Откуда берется балл протокола? Как считается?",
    "Типы ошибок в КК",
    "Авторизация: не получается авторизоваться, что делать? Пароль не подходит"
]

In [ ]:
from docx import Document
from docx.shared import Pt, Inches
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION
from docx.enum.table import WD_TABLE_ALIGNMENT
import os

# Инициализация документа
doc = Document()

# Добавление заголовка
doc.add_heading('Вопросы и ответы по тематике КК', level=1)

# Обработка каждого вопроса
for question in all_questions:
    # Получение ответа и списка фотографий
    answer, photos, _ = qa_system.answer_question(question, use_hyde=True)
    
    # Добавление параграфа с вопросом
    q_paragraph = doc.add_paragraph()
    q_run = q_paragraph.add_run(f"Вопрос: {question}")
    q_run.bold = True  # Жирный шрифт для вопроса
    
    # Добавление параграфа с ответом
    a_paragraph = doc.add_paragraph()
    a_run = a_paragraph.add_run(f"Ответ: {answer}")
    a_run.font.size = Pt(11)  # Размер шрифта для ответа
    a_run.font.name = 'Calibri'  # Шрифт для ответа
    a_paragraph.paragraph_format.left_indent = Inches(0.25)  # Отступ для ответа
    
    # Проверка наличия фотографий
    if photos:
        print(photos)
        for photo in photos:
            photo_path = photo.get('path').replace('\\', '/')
            photo_description = photo.get('description', 'Описание отсутствует')
            
            # Проверка существования файла фотографии
            if os.path.exists(photo_path):
                # Добавление фотографии
                try:
                    # Вставка изображения с заданной шириной (например, 4 дюйма)
                    doc.add_picture(photo_path, width=Inches(4))
                    
                    # Получение последнего добавленного параграфа (под фотографией)
                    last_paragraph = doc.paragraphs[-1]
                    
                    # Центрирование фотографии
                    last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    
                    # Добавление описания фотографии
                    desc_paragraph = doc.add_paragraph()
                    desc_run = desc_paragraph.add_run(f"Описание фотографии: {photo_description}")
                    desc_run.font.size = Pt(10)  # Размер шрифта для описания
                    desc_run.font.name = 'Calibri'  # Шрифт для описания
                    desc_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Центрирование описания
                except Exception as e:
                    print(f"Ошибка при вставке изображения {photo_path}: {e}")
            else:
                print(f"Файл фотографии не найден: {photo_path}")
    
    # Добавление пустой строки для разделения Q&A
    doc.add_paragraph("\n")

# Сохранение документа
doc.save("QA_all.docx")